In [94]:
from facenet_pytorch import MTCNN
from PIL import Image
from matplotlib import pyplot  as plt
import numpy as np
import math

In [95]:
frontal_angle_r_range = range(35, 57)  # Right eye angle range for frontal face
frontal_angle_l_range = range(35, 58)  # Left eye angle range for frontal face

In [96]:
mtcnn = MTCNN(
    image_size=160,
    margin=0,
    min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], # MTCNN thresholds
    factor=0.709,
    post_process=True,
    device='cpu'
)

In [97]:
def angle(a, b, c):
    ba = np.array(a) - np.array(b)
    bc = np.array(c) - np.array(b) 
    
    cosine_angle = np.dot(ba, bc)/(np.linalg.norm(ba)*np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    
    return np.degrees(angle)

In [100]:
def pred_face_pose(file_path):
    image = Image.open(file_path) # Reading the image
    
    if image.mode != "RGB":
        im = image.convert('RGB')
    
    bbox_, prob_, landmarks_ = mtcnn.detect(image, landmarks=True) # The detection part producing bounding box, probability of the detected face, and the facial landmarks
    
    if bbox_ is None: 
        return [{'error': 'error_no_face_detected'}]
    
    faces = []
    for bbox, landmarks, prob in zip(bbox_, landmarks_, prob_):
        # To check if we detect a face in the image
        if bbox is None: 
            faces.append({'error': 'error_no_face_detected'})
            continue
            
        angle_r = angle(landmarks[0], landmarks[1], landmarks[2])  # Calculate the right eye angle
        angle_l = angle(landmarks[1], landmarks[0], landmarks[2])  # Calculate the left eye angle
    
        if (int(angle_r) in frontal_angle_r_range) and (int(angle_l) in frontal_angle_l_range):
            pred_label = 'frontal'
        else: 
            if angle_r < angle_l:
                pred_label = 'left'
            else:
                pred_label = 'right'
                
        face = {
            'error': None,
            'bbox': bbox.tolist(),
            'landmarks': {
                'left_eye': landmarks[0].tolist(),
                'right_eye': landmarks[1].tolist(),
                'nose': landmarks[2].tolist(),
                'left_mouth': landmarks[3].tolist(),
                'right_mouth': landmarks[4].tolist()
            
            },
            'angle_r': angle_r,
            'angle_l': angle_l,
            'pred_prob': prob,
            'pred_label': pred_label
        }
                
        faces.append(face)
            
    return faces